## Process of Execution

Process of Execution:

- Run the 'Required Libraries' section

- Upload 'AID-a_file', 'AID-e_file', 'SDR', 'EON' year-wise datasets

- Run 'AID Data Analysis' section

- Run 'E-File AID Data Analysis' section

- Run 'SDR Data Analysis' section

- Run 'EON Data Analysis' section

- Run 'STM.csv to be used in R' section and upload the individual csv files of AID, SDR and EON to the working directory of R Code

- After running the R code, upload the csv files obtained for each dataset AID, SDR and EON and then run 'Merging of Keywords Column from R'

- Run 'Fuzzy Merging' section



## Required Libraries

In [ ]:
# Importing required libraries
import warnings
warnings.filterwarnings("ignore")

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import pandas as pd

import numpy as np

## AID Data Analysis

In [ ]:
# Read the 2019 AID Data a-file 
aid_data = pd.read_csv('a2019.csv',encoding='UTF-8',skipinitialspace=True)

aid_data1 = aid_data

# Data Cleaning and structuring
aid_data1.columns= aid_data1.columns.str.strip()

# aid_data1 = aid_data1.dropna(subset=['c5'],how='all')
# Drop the last two columns (Extra empty columns exists at the end of the file)
aid_data1 = aid_data1.drop(aid_data1.columns[[-1,-2]],axis=1)

# Check the columns c5 for any character string (This column should not contain text)
fil = aid_data1['c5'].str.contains(r'^[a-zA-Z]')

aid_data1 = aid_data1[~fil]

# Resetting the Index Values
aid_data1.reset_index(drop=True,inplace=True)

# Unifying the date formats 
aid_data1['c9'] = pd.to_datetime(aid_data1['c9'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c205'] = pd.to_datetime(aid_data1['c205'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c206'] = pd.to_datetime(aid_data1['c206'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c207'] = pd.to_datetime(aid_data1['c207'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c208'] = pd.to_datetime(aid_data1['c208'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c210'] = pd.to_datetime(aid_data1['c210'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c229'] = pd.to_datetime(aid_data1['c229'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c230'] = pd.to_datetime(aid_data1['c230'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c10'] = pd.to_datetime(aid_data1['c10'].astype(str),format= '%H%M.0',errors='coerce').dt.strftime('%H:%M')

# Making some columns as integer type for further analysis(most of these are the ones which provide the information about the number of people injured or killed on the aircraft)
aid_data1[aid_data1.columns[79:95]] = aid_data1[aid_data1.columns[79:95]].fillna(0)

for col in aid_data1.columns[80:93].values:
    aid_data1[col] = aid_data1[col].astype('int64')

#replacing the columns that has zero to NAN 
aid_data1[['c149','c151','c145']] = aid_data1[['c149','c151','c145']].fillna(0).astype(int).astype(str).replace('0','NaN')

# Renaming important columns and adding suffix to other columns to distinguish the data of one report from another to understand better for further analysis
aid_data1.rename(columns={'c9': 'Date_Of_Occurrence', 'c22': 'N_Number'}, inplace=True)

aid_data1 = aid_data1.add_suffix('_aid').rename(columns={'Report_Type_aid': 'Report_Type', 'Date_Of_Occurrence_aid': 'Date_Of_Occurrence'})

aid_data1 = aid_data1.rename(columns={'c5_aid': 'Unique_Identifier'})

# Inserting a new column of report type to help further analysis
aid_data1.insert(0,'Report_Type','AID')
    
# aid_data1[aid_data1.columns[79:95]]

In [ ]:
#Top 10 records in the dataset
aid_data1.head(10)

,Report_Type,Unique_Identifier,c1_aid,c2_aid,c3_aid,c4_aid,c6_aid,c7_aid,c8_aid,Date_Of_Occurrence,c10_aid,c75_aid,c132_aid,c134_aid,c136_aid,c138_aid,c139_aid,c140_aid,c141_aid,c144_aid,c145_aid,c147_aid,c149_aid,c151_aid,c152_aid,c153_aid,c155_aid,c157_aid,c160_aid,c162_aid,c203_aid,c204_aid,c214_aid,c233_aid,c234_aid,c790_aid,N_Number_aid,c23_aid,c24_aid,c25_aid,...,c192_aid,c244_aid,c40_aid,c44_aid,c46_aid,c48_aid,c51_aid,c77_aid,c79_aid,c81_aid,c83_aid,c85_aid,c87_aid,c89_aid,c91_aid,c93_aid,c95_aid,c97_aid,c99_aid,c101_aid,c103_aid,c105_aid,c107_aid,c109_aid,c131_aid,c133_aid,c135_aid,c137_aid,c146_aid,c148_aid,c150_aid,c154_aid,c156_aid,c158_aid,c161_aid,c163_aid,c183_aid,c191_aid,c229_aid,c230_aid
0,AID,20191013023229A,A,91,NaN,NaN,2019,10,13,2019-10-13,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CV,NaN,NaN,224306,2322,NaN,NaN,NaN,NaN,96709,TAYLOR,BC12-D,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Conventional,NaN,NaN,NaN,NaN,1950-07-28,NaT
1,AID,20191122023189A,A,91,NaN,NaN,2019,11,22,2019-11-22,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224301,2318,NaN,NaN,NaN,NaN,969JM,CIRRUS,SR22T,NaN,...,NaN,NaN,COMM PILOT FLIGHT IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-07-29,NaT
2,AID,20191125023199I,I,91,NaN,NaN,2019,11,25,2019-11-25,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224302,2319,NaN,NaN,NaN,NaN,3201L,MOONEY,M20J,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Other, Specify",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1961-12-21,NaT
3,AID,20191217023209A,A,91,NaN,NaN,2019,12,17,2019-12-17,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224303,2320,NaN,NaN,NaN,NaN,3113V,BEECH,35R,NaN,...,NaN,NaN,COMMERCIAL PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taxi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1933-11-17,NaT
4,AID,20191218023219I,I,91,NaN,NaN,2019,12,18,2019-12-18,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,224304,2321,NaN,NaN,NaN,NaN,6919J,PIPER,PA-28-151,NaN,...,NaN,NaN,PRIVATE PILOT FLT IN,NaN,NaN,NaN,NaN,6111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,2000-02-19,2001-06-13
5,AID,20191101023269A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224310,2326,NaN,NaN,NaN,NaN,102PH,HATFIE,CH701,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1950-03-10,NaT
6,AID,20191101023289A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224312,2328,NaN,NaN,NaN,NaN,732JW,CESSNA,T210L,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-02-27,NaT
7,AID,20191031023279A,A,91,NaN,NaN,2019,10,31,2019-10-31,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L72,1,L,7,2,3O,NaN,NaN,TR,NaN,NaN,224311,2327,NaN,NaN,NaN,NaN,959CM,RAYTHE,58,NaN,...,NaN,NaN,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Landing: Approach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN

In [ ]:
#Writing the modified dataframe to a CSV
# aid_data1.to_csv('a2019_Clean.csv', index=False)

## E-File AID Data Analysis

In [ ]:
import more_itertools as mit
# Read the 2019 AID Data e-file 

e_aid_data = pd.read_csv('e2019(possible).csv',encoding='UTF-8',skipinitialspace=True)

e_aid_data1= e_aid_data

# Dropping blank rows from primary key column of c5
e_aid_data1 = e_aid_data1.dropna(subset=['c5'],how='all')

# Combining all the text data into one separate column and dropping the individual text columns
e_aid_data1['remark'] = e_aid_data1[['remark','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9']].replace('?','').fillna('').agg(' '.join, axis=1)
e_aid_data1 = e_aid_data1.drop(e_aid_data1.columns[[2,3,4,5,6,7,8,9]],axis=1)

e_aid_data1.reset_index(drop=True,inplace=True)

e_aid_data1['remark']= e_aid_data1['remark'].replace(r'^\s*$', np.nan, regex=True) #replacing extra whitespace
e_aid_data1['remark']= e_aid_data1['remark'].mask(pd.isnull, e_aid_data1['c5']) #

fill = e_aid_data1['c5'].str.contains(r'^\b2019')

e_aid_data1['c5'][fill==False] = np.nan

#Unifying all the related text remarks corresponding to each unique identifier of AID
q=[index for index, row in e_aid_data1.iterrows() if row.isnull().any()]

q=[list(group) for group in mit.consecutive_groups(q)] 

for i in q:
    t=''
    s=i[0]
    for j in i:
        t+=(e_aid_data1.iloc[j]['remark'])
        t+=' '
    e_aid_data1.iloc[s-1]['remark']+=t
        
e_aid_data1 = e_aid_data1[pd.notnull(e_aid_data1['c5'])]

e_aid_data1.reset_index(drop=True,inplace=True)

# Renaming the columns for better understanding
e_aid_data1.rename(columns={'c5':'c5_aid','remark': 'Remarks'},inplace=True)

e_aid_data1 = e_aid_data1.rename(columns={'c5_aid': 'Unique_Identifier'})

# Part of text preprocessing of remarks column
e_aid_data1['Remarks'] = e_aid_data1['Remarks'].str.replace('\^PRIVACY DATA OMITTED\^', '').str.strip().str.replace('\s+', ' ', regex=True)

e_aid_data1['Remarks'] = e_aid_data1['Remarks'].str.replace('(#NAME\?)', '').str.strip().str.replace('\s+', ' ', regex=True)

e_aid_data1.insert(0,'Report_Type','AID') #inserting a column which represents the type of report

e_aid_data1

,Report_Type,Unique_Identifier,Remarks
0,AID,20190101007219I,DURING APPROACH TO STUART WITHAM AIRPORT THE A...
1,AID,20190101007229I,THE ONLY INFORMATION PROVIDED BY AIRMAN WAS TH...
2,AID,20190101008169I,"ON DEPARTURE ROLL FROM PORT EADS GRASS STRIP, ..."
3,AID,20190101016219I,"VENICE, FL. N241EE, BE35 LANDED ON RUNWAY 23 A..."
4,AID,20190102000019A,1/2/19 AT 1045 EST. N772MG AN R44 HELICOPTER C...
...,...,...,...
2403,AID,20191231019739I,PILOT WAS IN THE TRAFFIC PATTERN FOR THE CREST...
2404,AID,20191231019749I,PILOT WAS CONDUCTING A FAR PART 91 TRIP TO MCC...
2405,AID,20191231019759A,PILOT UTILIZING PRIVATE MOWED STRIP TO PRACTIC...
2406,AID,20191231020669A,THE PILOT WITH 3 PASSENGERS INITIATED A TAKEOF...


In [ ]:
# e_aid_data1.to_csv('e2019(possible)_Clean.csv', index=False)

In [ ]:
# Merging of e-file of AID with the a-file of AID to have the full remarks column in one dataset
aid_data1 = pd.merge(aid_data1, e_aid_data1, how="inner", on=["Report_Type", "Unique_Identifier"])

# Dropping C119 column
aid_data1 = aid_data1.drop('c119_aid',axis=1)

#Replacing missing values in Phase of flight column with 'Unknown'
aid_data1['c95_aid'] = aid_data1['c95_aid'].fillna('Unknown')

#Replacing missing values in Aircraft Make column with 'Unknown'
aid_data1['c23_aid']= aid_data1['c23_aid'].fillna('Unknown')

#Replacing missing values in Aircraft Model column with 'Aircraft'
aid_data1['c24_aid']= aid_data1['c24_aid'].fillna('Aircraft')

#Concatenating Aircraft Make and Model to a single column called Aircraft Type
aid_data1['Aircraft_Type'] = aid_data1['c23_aid'] + ' ' + aid_data1['c24_aid']
aid_data1['Aircraft_Type'] = aid_data1['Aircraft_Type'].str.strip().str.replace('\s+', ' ', regex=True).astype(str)

#Renaming as per convenience
aid_data1 = aid_data1.rename(columns={'c95_aid':'Phase_Of_Flight','c23_aid':'Aircraft_Make'})

#Removing the extra spaces to the left and right of the data in Phase of Flight column
aid_data1['Phase_Of_Flight'] = aid_data1['Phase_Of_Flight'].str.strip()

#Top 10 rows in AID
aid_data1.head(10)

,Report_Type,Unique_Identifier,c1_aid,c2_aid,c3_aid,c4_aid,c6_aid,c7_aid,c8_aid,Date_Of_Occurrence,c10_aid,c75_aid,c132_aid,c134_aid,c136_aid,c138_aid,c139_aid,c140_aid,c141_aid,c144_aid,c145_aid,c147_aid,c149_aid,c151_aid,c152_aid,c153_aid,c155_aid,c157_aid,c160_aid,c162_aid,c203_aid,c204_aid,c214_aid,c233_aid,c234_aid,c790_aid,N_Number_aid,Aircraft_Make,c24_aid,c25_aid,...,c40_aid,c44_aid,c46_aid,c48_aid,c51_aid,c77_aid,c79_aid,c81_aid,c83_aid,c85_aid,c87_aid,c89_aid,c91_aid,c93_aid,Phase_Of_Flight,c97_aid,c99_aid,c101_aid,c103_aid,c105_aid,c107_aid,c109_aid,c131_aid,c133_aid,c135_aid,c137_aid,c146_aid,c148_aid,c150_aid,c154_aid,c156_aid,c158_aid,c161_aid,c163_aid,c183_aid,c191_aid,c229_aid,c230_aid,Remarks,Aircraft_Type
0,AID,20191013023229A,A,91,NaN,NaN,2019,10,13,2019-10-13,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CV,NaN,NaN,224306,2322,NaN,NaN,NaN,NaN,96709,TAYLOR,BC12-D,NaN,...,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Conventional,NaN,NaN,NaN,NaN,1950-07-28,NaT,THE PILOT OBTAINED LOCAL AUTOMATED WEATHER REP...,TAYLOR BC12-D
1,AID,20191122023189A,A,91,NaN,NaN,2019,11,22,2019-11-22,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224301,2318,NaN,NaN,NaN,NaN,969JM,CIRRUS,SR22T,NaN,...,COMM PILOT FLIGHT IN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-07-29,NaT,DEPARTED TRL FOR FLIGHT TO GLADEWATER. CLIMBED...,CIRRUS SR22T
2,AID,20191125023199I,I,91,NaN,NaN,2019,11,25,2019-11-25,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224302,2319,NaN,NaN,NaN,NaN,3201L,MOONEY,M20J,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Other, Specify",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1961-12-21,NaT,"ON NOVEMBER 25, 2019, AT APPROXIMATELY 1130 EA...",MOONEY M20J
3,AID,20191217023209A,A,91,NaN,NaN,2019,12,17,2019-12-17,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224303,2320,NaN,NaN,NaN,NaN,3113V,BEECH,35R,NaN,...,COMMERCIAL PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taxi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1933-11-17,NaT,THE PILOT/OWNER WAS INTENDING TO MAKE A SHORT ...,BEECH 35R
4,AID,20191218023219I,I,91,NaN,NaN,2019,12,18,2019-12-18,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,224304,2321,NaN,NaN,NaN,NaN,6919J,PIPER,PA-28-151,NaN,...,PRIVATE PILOT FLT IN,NaN,NaN,NaN,NaN,6111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,2000-02-19,2001-06-13,A PORTION OF THE PROPELLER SEPARATED FROM AIRC...,PIPER PA-28-151
5,AID,20191101023269A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224310,2326,NaN,NaN,NaN,NaN,102PH,HATFIE,CH701,NaN,...,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1950-03-10,NaT,"ON FRIDAY, NOVEMBER 1, 2019, A ZENITH CH701, R...",HATFIE CH701
6,AID,20191101023289A,A,91,NaN,NaN,2019,11,1,2019-11-01,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1H71,1,H,7,1,3O,NaN,NaN,TR,NaN,NaN,224312,2328,NaN,NaN,NaN,NaN,732JW,CESSNA,T210L,NaN,...,PRIVATE PILOT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-HIGH WING/PARA WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-02-27,NaT,"ON NOVEMBER 1, 2019, AT 1350 CDT, A CESS

In [ ]:
#Dimensions of the Dataset
aid_data1.shape

(2391, 181)

In [ ]:
# aid_data1.to_csv('AID_a-e_file_combined.csv', index=False)

## SDR Data Analysis

In [ ]:
# Read the 2019 SDR Data file 

sdr_data = pd.read_csv('sdr(2019_Occurred_Incidents).csv',encoding='UTF-8',skipinitialspace=True,dtype=str)
sdr_data1 = sdr_data

# Dropping blank rows (if any) from the main column of c25
sdr_data1 = sdr_data1.dropna(subset=['c25'],how='all')

# Dropping columns 
sdr_data1 = sdr_data1.drop(sdr_data1.columns[[-1,-2,-3,0]],axis=1)
#Removing of spaces
sdr_data1.columns= sdr_data1.columns.str.strip()

#changing the date format into Year/month/day
sdr_data1['c10'] = pd.to_datetime(sdr_data1['c10'].astype(str), format='%Y%m%d',errors='coerce')
sdr_data1['c25'] = pd.to_datetime(sdr_data1['c25'].astype(str), format='%Y%m%d',errors='coerce')

# Joining the Remarks columns
sdr_data1['Remarks'] = sdr_data1[['c510a','c510b','c510c','c510d','c510e','c510f']].fillna('').agg(''.join, axis=1)

# Removing extra whitespace
sdr_data1['Remarks']= sdr_data1['Remarks'].str.strip().str.replace('\s+', ' ', regex=True)

#dropping the duplicate columns 
sdr_data1.drop(['c510a', 'c510b','c510c','c510d','c510e','c510f'], axis=1, inplace=True)

#Renaming the columns
sdr_data1.rename(columns={'c25': 'Date_Of_Occurrence', 'c390': 'N_Number'}, inplace=True)

#Adding sdr as suffix
sdr_data1 = sdr_data1.add_suffix('_sdr').rename(columns={'Report_Type_sdr': 'Report_Type', 'Date_Of_Occurrence_sdr': 'Date_Of_Occurrence','N_Number_sdr':'N_Number','Remarks_sdr':'Remarks'})

sdr_data1['c18_sdr'] = sdr_data1['c18_sdr'].str.replace('\W', '')

#Renaming the c18 column as Unique Identifier
sdr_data1 = sdr_data1.rename(columns={'c18_sdr': 'Unique_Identifier'})

#Filling null values of Aircraft Make column with 'Unknown'
sdr_data1['c130_sdr']=sdr_data1['c130_sdr'].fillna('Unknown')

#Filling null values of Aircraft Model column with 'Unknown'
sdr_data1['c140_sdr']=sdr_data1['c140_sdr'].fillna('Aircraft')

sdr_data1['c100_sdr'] = sdr_data1['c100_sdr'].str.strip()

sdr_data1['c250_sdr'] = sdr_data1['c250_sdr'].str.strip()

sdr_data1['c260_sdr'] = sdr_data1['c260_sdr'].str.strip()

sdr_data1['c100_sdr']=sdr_data1['c100_sdr'].fillna('UNKNOWN')

sdr_data1['c250_sdr']=sdr_data1['c250_sdr'].fillna('UNKNOWN')

sdr_data1['c260_sdr']=sdr_data1['c260_sdr'].fillna('UNKNOWN')

sdr_data1['Aircraft_Type'] = sdr_data1['c130_sdr'] + ' '+ sdr_data1['c140_sdr']
sdr_data1['Aircraft_Type'] = sdr_data1['Aircraft_Type'].str.strip().str.replace('\s+', ' ', regex=True).astype(str)

sdr_data1['Remarks'] = sdr_data1['Remarks'].str.replace('\x1a', '')

sdr_data1.rename(columns={'c332_sdr':'Stage_Of_Operation','c100_sdr':'Description_Of_Defective_Part','c250_sdr':'Location_Of_Defective_Part','c260_sdr':'Condition_Of_Defective_Part','c130_sdr':'Aircraft_Make'}, inplace=True)

sdr_data1['Stage_Of_Operation'] = sdr_data1['Stage_Of_Operation'].str.strip()


sdr_data1.insert(0,'Report_Type','SDR')

# print the first 5 rows
sdr_data1.head(5)

,Report_Type,c10_sdr,c12_sdr,c14_sdr,c15_sdr,c16_sdr,c17_sdr,Unique_Identifier,c20_sdr,Date_Of_Occurrence,c35_sdr,c40_sdr,c90_sdr,Description_Of_Defective_Part,c110_sdr,c120_sdr,Aircraft_Make,c140_sdr,c150_sdr,c152_sdr,c160_sdr,c170_sdr,c180_sdr,c190_sdr,c192_sdr,c200_sdr,c210_sdr,c220_sdr,c230_sdr,c240_sdr,Location_Of_Defective_Part,Condition_Of_Defective_Part,c270_sdr,c280_sdr,c290_sdr,c300_sdr,c310a_sdr,c310b_sdr,c310c_sdr,c310d_sdr,...,c314b_sdr,c314c_sdr,c314d_sdr,c320a_sdr,c320b_sdr,c320c_sdr,c324a_sdr,c324b_sdr,c324c_sdr,c330_sdr,Stage_Of_Operation,c340_sdr,c350_sdr,c360_sdr,c370_sdr,c380_sdr,N_Number,c400_sdr,c410_sdr,c420_sdr,c430_sdr,c440_sdr,c450_sdr,c460_sdr,c490_sdr,c602_sdr,c604_sdr,c606_sdr,c608_sdr,c610_sdr,c612_sdr,c614_sdr,c616_sdr,c620_sdr,c640_sdr,c652_sdr,c654_sdr,c660_sdr,Remarks,Aircraft_Type
0,SDR,2019-09-06,NaN,NM,2019,9,6,FDEA201904040764,1,2019-04-04,A,5320,ACA314011,FLOOR SUPPORT,NaN,NaN,DOUG,MD11,MD11F,3023600,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZONE 200,CORRODED,A,NaN,NaN,FDEA,K,NaN,NaN,NaN,...,NaN,NaN,NaN,O,NaN,NaN,OTHER,NaN,NaN,IN,INSP/MAINT,1,NaN,NaN,CE,23,604FE,NaN,NaN,NaN,NaN,48460,NaN,NaN,NaN,NaN,NaN,NaN,3,4F,NaN,NaN,RT,A22WE,NaN,NaN,NaN,NaN,FLOOR BOARD SUPPORT CORRODED @ STA. 6-70 CENTE...,DOUG MD11
1,SDR,2019-11-18,NaN,NE,2019,11,18,11FA2019070100001,2,2019-05-23,A,7200,NaN,ENGINE,NaN,NaN,CESSNA,510,510,2076617,CE,PWC,PW615,PW615FA,52282,NE,NaN,NaN,NaN,NaN,NR 2,MALFUNCTIONED,E,NaN,NaN,11FA,C,NaN,NaN,NaN,...,NaN,NaN,NaN,M,J,NaN,OVER TEMP,WARNING INDICATION,NaN,TO,TAKEOFF,1,NaN,NaN,SW,3,820UT,NaN,NaN,NaN,NaN,5100332,LB0666,NaN,NaN,NaN,NaN,NaN,2,4F,4,F,RT,A00014WI,E00073EN,NaN,NaN,NaN,ITT SPIKE NR 2 ENGINE EXCEEDED REDLINE. ENGINE...,CESSNA 510
2,SDR,2019-11-18,NaN,CE,2019,11,18,11FA2019070100002,1,2019-05-20,A,7722,NaN,UNKNOWN,NaN,NaN,CESSNA,510,510,2076617,CE,PWC,PW615,PW615FA,52282,NE,NaN,NaN,NaN,NaN,UNKNOWN,MALFUNCTIONED,E,NaN,NaN,11FA,K,NaN,NaN,NaN,...,NaN,NaN,NaN,J,M,NaN,WARNING INDICATION,OVER TEMP,NaN,CL,CLIMB,1,NaN,NaN,SW,3,820UT,NaN,NaN,NaN,NaN,5100332,LB0666,NaN,NaN,NaN,NaN,NaN,2,4F,4,F,RT,A00014WI,E00073EN,NaN,NaN,NaN,ITT SPIKE IN THE CLIMB FOR UNKNOWN REASONS. MA...,CESSNA 510
3,SDR,2020-03-24,NaN,NM,2020,3,24,120B201911200001,1,2019-11-18,A,2697,5534001R16,WIRE HARNESS,KIDDE,NA,BOEING,767,767323,1385205,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT FIRE LOOP,BROKEN,I,NaN,NaN,120B,K,NaN,NaN,NaN,...,NaN,NaN,NaN,O,NaN,NaN,OTHER,NaN,NaN,IN,INSP/MAINT,1,NaN,NaN,GL,3,36NE,NaN,NaN,NaN,NSN,25193,NaN,NaN,NaN,NaN,NaN,NaN,2,4F,NaN,NaN,RT,A1NM,NaN,NaN,NaN,NaN,RH ENGINE FIRE LOOP 1 DISPLAYED ON EICAS ON PR...,BOEING 767
4,SDR,2020-06-02,NaN,NM,2020,6,2,120B201912240002,1,2019-12-23,A,2841,2021207,INDICATOR,NaN,NaN,BOEING,767,767323,1385205,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FUEL QTY,INOPERATIVE,I,NaN,NaN,120B,H,NaN,NaN,NaN,...,NaN,NaN,NaN,J,NaN,NaN,WARNING INDICATION,NaN,NaN,NR,NOT REPORTED,1,NaN,NaN,GL,3,225NE,99331,16,NaN,NaN,25194,NaN,NaN,NaN,NaN,NaN,NaN,2,4F,NaN,NaN,RT,A1NM,NaN,NaN,NaN,NaN,FUEL QUANTITY SYSTEM DISPLAYED ON PREFLIGHT EI...,BOEING 767


In [ ]:
#Dimensions of the Dataset
sdr_data1.shape

(61009, 81)

In [ ]:
# sdr_data1.to_csv('sdr2019_Clean.csv', index=False)

## EON Data Analysis

In [ ]:
# Read the 2019 EON Data file 

eon_data = pd.read_csv('EON-2019.csv',encoding='UTF-8',skipinitialspace=True)
eon_data1 = eon_data

# eon_data1 = eon_data1.dropna(axis='columns', how='all')
#Removing the spaces
eon_data1.columns= eon_data1.columns.str.strip()

#Changing the event date to date and time format 
eon_data1['Event Date'] = pd.to_datetime(eon_data1['Event Date'])
eon_data1['Event Time'] = eon_data1['Event Date'].dt.strftime('%H:%M')
eon_data1['Event Date'] = eon_data1['Event Date'].dt.strftime('%Y-%m-%d')

eon_data1['Event Date'] = pd.to_datetime(eon_data1['Event Date'])

cols = eon_data1.columns.tolist()
cols = cols[0:5]+cols[-1:]+cols[5:36]
eon_data1 = eon_data1[cols]

# joining the Remarks column  
eon_data1['Message Whole'] = eon_data1[['MessageText.1','Unnamed: 29','Unnamed: 30','Unnamed: 31','Unnamed: 32','Unnamed: 33','Unnamed: 34','Unnamed: 35']].fillna('').agg(''.join, axis=1)

eon_data1['Message Whole'] = eon_data1['Message Whole'].str.strip().str.replace('\s+', ' ', regex=True)

#renaming the columns
eon_data1.rename(columns={'Message Whole': 'Remarks','Event Date': 'Date_Of_Occurrence'},inplace=True)

cols1 = eon_data1.columns.tolist()
cols1 = cols1[0:29] + cols1[-1:]

eon_data1 = eon_data1[cols1]

#Extracting the possible N-Number from Title column and made it as a seperate column
eon_data1['N_Number'] = eon_data1['Title'].str.extract(r'((?<=\|).+?(?=\|))')

#Removing the spaces
eon_data1['N_Number'] = eon_data1['N_Number'].str.strip()

fil = eon_data1['N_Number'].str.contains('^N[0-9]|^n[0-9]')

# Replacing the 'n' to N
eon_data1['N_Number'][fil==True] = eon_data1['N_Number'][fil==True].apply(lambda x : x[1:] if str(x).startswith("N") else x).apply(lambda x : x[1:] if str(x).startswith("n") else x)

#Renaming ID column with Unique Identifier
eon_data1 = eon_data1.rename(columns={'Id': 'Unique_Identifier'})

#Removing Left spaces in the remarks column
eon_data1.Remarks = eon_data1.Remarks.str.lstrip(',').str.strip()
#Dropping of Duplicates in Unique Identifier
eon_data1 = eon_data1.drop_duplicates(subset=['Unique_Identifier'])
#Resetting of the index
eon_data1.reset_index(drop=True,inplace=True)

# Checking whether Aircraft type is present in the remarks if present then Remove the aircraft type from the Remarks
eon_data1['Aircraft Type'][eon_data1['Aircraft Type'].isna()] = ''
for i in range(len(eon_data1)):
  temp = eon_data1.loc[i,'Aircraft Type']
  if temp in eon_data1.loc[i,'Remarks']:
    eon_data1.loc[i,'Remarks'] = eon_data1.loc[i,'Remarks'].replace(temp,'')
    eon_data1.loc[i,'Remarks'] = eon_data1.loc[i,'Remarks'].replace(',','').strip()

#Blank Remarks is replaced with Not Reported
eon_data1.Remarks[eon_data1.Remarks == ''] = 'NOT REPORTED'

#Replacing Blank and Unknown values in Aircraft Type column to hold a single string 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']==''] = 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']=='UNKN'] = 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']=='TYPE UNKNOWN'] = 'UNKNOWN'
eon_data1['Aircraft Type'][eon_data1['Aircraft Type']=='UNKNOWN TYPE'] = 'UNKNOWN'

eon_data1 = eon_data1.rename(columns={'Event Type':'Event_Type','Aircraft Type':'Aircraft_Type'})

eon_data1.insert(0,'Report_Type','EON')

# eon_data1.drop(['Title'], axis=1, inplace=True)
#Print the first 5 rows
eon_data1.head(5)

,Report_Type,Unique_Identifier,Title,Description,Event_Type,Date_Of_Occurrence,Event Time,Reported By,Received Date,Latitude,Longitude,City,State,Airport,Region,Reporting Facility,Aircraft_Type,Departure,Arrival,Diverted To,POB,Injury Type,Injuries Minor,Injuries Serious,Fatalities,Injuries Unknown,Uninjured,POB Unknown,Damage Type,MessageText,Remarks,N_Number
0,EON,554891,MEDICAL EMERGENCY - PASSENGER | ASA1305 | SAN ...,NaN,MEDICAL EMERGENCY - PASSENGER,2019-01-01,02:01,NaN,NaN,37.618805,-122.375420,SAN FRANCISCO,CA,SFO,AWP,MOR/SFO @,A320,NaN,SFO,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"SAN FRANCISCO, CA (SFO)",SFODECLARED A MEDICAL EMERGENCY FOR A 29 YEAR ...,ASA1305
1,EON,554892,GENERAL AVIATION OCCURRENCE | N9118C | SANTA R...,NaN,GENERAL AVIATION OCCURRENCE,2019-01-01,02:38,NaN,NaN,38.509693,-122.812889,SANTA ROSA,CA,STS,AWP,MOR/STS @ 1843 PST,C182,NaN,STS,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"SANTA ROSA, CA (STS)",STSLANDED WITHOUT INCIDENT AFTER REPORTING FAU...,9118C
2,EON,554894,GENERAL AVIATION OCCURRENCE | N613PJ | TETERBO...,NaN,GENERAL AVIATION OCCURRENCE,2019-01-01,03:15,NaN,NaN,40.850113,-74.060837,TETERBORO,NJ,TEB,AEA,TEB,CL60,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"TETERBORO, NJ (TEB)",DIVERTED TO TEB AND LANDED WITHOUT INCIDENT AF...,613PJ
3,EON,554895,"LASER EVENT | N4203P | PASCO, WA (PSC) 01-01-2...",NaN,LASER EVENT,2019-01-01,03:26,NaN,NaN,46.264694,-119.119026,PASCO,WA,PSC,ANM,MOR/PSC @ 1943 PST,C172,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"PASCO, WA (PSC)",WAS ILLUMINATED BY A GREEN LASER FROM THE 12 O...,4203P
4,EON,554908,AIR CARRIER OR TAXI OCCURRENCE | NKS8618 | DET...,NaN,AIR CARRIER OR TAXI OCCURRENCE,2019-01-01,11:25,NaN,NaN,42.212444,-83.353386,DETROIT,MI,DTW,AGL,DTW MOR,A320,DTW,DFW,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"DETROIT, MI (DTW)",DTW-DFW ABORTED TAKEOFF DUE TO A MALFUNCTION I...,NKS8618


In [ ]:
#Dimensions of the Dataset
eon_data1.shape

(40465, 32)

In [ ]:
# eon_data1.to_csv('EON-2019(Jan-Mar)_Clean.csv', index=False)

# STM.csv to be used in R

In [ ]:
#Selecting some of the columns in AID Dataset and assign to AID STM
aid_stm = aid_data1[['Unique_Identifier','Date_Of_Occurrence','Aircraft_Make','Phase_Of_Flight','Remarks']]
aid_stm.to_csv('aid_stm.csv', index=False)

#Selecting some of the columns in SDR Dataset and assign to SDR STM
sdr_stm = sdr_data1[['Unique_Identifier','Date_Of_Occurrence','Aircraft_Make','Stage_Of_Operation','Remarks']]
sdr_stm.to_csv('sdr_stm.csv', index=False)

#Selecting some of the columns in EON Dataset and assign to EON STM
eon_stm = eon_data1[['Unique_Identifier','Date_Of_Occurrence','Aircraft_Type','Remarks']]
eon_stm.to_csv('eon_stm.csv', index=False)

# Merging of Keywords Column from R

In [ ]:
# Structural Topic Model Result from R 
#Reading the STM Aid from R with Unique Identifier and Dominant words
final_aid_topic = pd.read_csv('final_aid_topic.csv',encoding='UTF-8',skipinitialspace=True)
final_aid_topic.columns = final_aid_topic.columns.str.strip().str.replace('\s+', ' ', regex=True)

#Reading the STM SDR from R with Unique Identifier and Dominant words
final_sdr_topic = pd.read_csv('final_sdr_topic.csv',encoding='UTF-8',skipinitialspace=True)
final_sdr_topic.columns = final_sdr_topic.columns.str.strip().str.replace('\s+', ' ', regex=True)

#Reading the STM EON from R with Unique Identifier and Dominant words
final_eon_topic = pd.read_csv('final_eon_topic.csv',encoding='UTF-8',skipinitialspace=True)
final_eon_topic.columns = final_eon_topic.columns.str.strip().str.replace('\s+', ' ', regex=True)

In [ ]:
# Merging of AID data and STM AId data file with common column Unique Identifier
aid_data1 = pd.merge(aid_data1, final_aid_topic, how="inner", on=["Unique_Identifier"])

# Merging of SDR data and STM Sdr data file with common column Unique Identifier
sdr_data1 = pd.merge(sdr_data1, final_sdr_topic, how="inner", on=["Unique_Identifier"])

# Merging of EON data and STM EON data file with common column Unique Identifier
eon_data1 = pd.merge(eon_data1, final_eon_topic, how="inner", on=["Unique_Identifier"])

In [ ]:
aid_data1.rename(columns={'Dominant_Topic_aid': 'Keywords_aid'},inplace=True)
#Top 5 records of the final AID dataset
aid_data1.head()

,Report_Type,Unique_Identifier,c1_aid,c2_aid,c3_aid,c4_aid,c6_aid,c7_aid,c8_aid,Date_Of_Occurrence,c10_aid,c75_aid,c132_aid,c134_aid,c136_aid,c138_aid,c139_aid,c140_aid,c141_aid,c144_aid,c145_aid,c147_aid,c149_aid,c151_aid,c152_aid,c153_aid,c155_aid,c157_aid,c160_aid,c162_aid,c203_aid,c204_aid,c214_aid,c233_aid,c234_aid,c790_aid,N_Number_aid,Aircraft_Make,c24_aid,c25_aid,...,c44_aid,c46_aid,c48_aid,c51_aid,c77_aid,c79_aid,c81_aid,c83_aid,c85_aid,c87_aid,c89_aid,c91_aid,c93_aid,Phase_Of_Flight,c97_aid,c99_aid,c101_aid,c103_aid,c105_aid,c107_aid,c109_aid,c131_aid,c133_aid,c135_aid,c137_aid,c146_aid,c148_aid,c150_aid,c154_aid,c156_aid,c158_aid,c161_aid,c163_aid,c183_aid,c191_aid,c229_aid,c230_aid,Remarks,Aircraft_Type,Keywords_aid
0,AID,20191013023229A,A,91,NaN,NaN,2019,10,13,2019-10-13,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CV,NaN,NaN,224306,2322,NaN,NaN,NaN,NaN,96709,TAYLOR,BC12-D,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Takeoff: Climb Out,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Conventional,NaN,NaN,NaN,NaN,1950-07-28,NaT,THE PILOT OBTAINED LOCAL AUTOMATED WEATHER REP...,TAYLOR BC12-D,"veered,ditch,grass,onto,brakes,wind,crosswind"
1,AID,20191122023189A,A,91,NaN,NaN,2019,11,22,2019-11-22,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224301,2318,NaN,NaN,NaN,NaN,969JM,CIRRUS,SR22T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1967-07-29,NaT,DEPARTED TRL FOR FLIGHT TO GLADEWATER. CLIMBED...,CIRRUS SR22T,"tanks,quit,restart,carburetor,switched,fuel,se..."
2,AID,20191125023199I,I,91,NaN,NaN,2019,11,25,2019-11-25,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224302,2319,NaN,NaN,NaN,NaN,3201L,MOONEY,M20J,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Other, Specify",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1961-12-21,NaT,"ON NOVEMBER 25, 2019, AT APPROXIMATELY 1130 EA...",MOONEY M20J,"field,meteorological,registered,visual,substan..."
3,AID,20191217023209A,A,91,NaN,NaN,2019,12,17,2019-12-17,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224303,2320,NaN,NaN,NaN,NaN,3113V,BEECH,35R,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Taxi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDER 12501 LBS,MONOPLANE-LOW WING,POWERED,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,1933-11-17,NaT,THE PILOT/OWNER WAS INTENDING TO MAKE A SHORT ...,BEECH 35R,"fire,smoke,extinguished,mercy,start,husband,co..."
4,AID,20191218023219I,I,91,NaN,NaN,2019,12,18,2019-12-18,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,224304,2321,NaN,NaN,NaN,NaN,6919J,PIPER,PA-28-151,NaN,...,NaN,NaN,NaN,NaN,6111.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cruise/Level Flight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wheeled-Tricycle,NaN,NaN,NaN,NaN,2000-02-19,2001-06-13,A PORTION OF THE PROPELLER SEPARATED FROM AIRC...,PIPER PA-28-151,"terrain,fatally,circumstances,destroyed,fatal,..."


In [ ]:
sdr_data1.rename(columns={'Dominant_Topic_sdr': 'Keywords_sdr'},inplace=True)
#Top 5 records of the final SDR dataset
sdr_data1.head()

,Report_Type,c10_sdr,c12_sdr,c14_sdr,c15_sdr,c16_sdr,c17_sdr,Unique_Identifier,c20_sdr,Date_Of_Occurrence,c35_sdr,c40_sdr,c90_sdr,Description_Of_Defective_Part,c110_sdr,c120_sdr,Aircraft_Make,c140_sdr,c150_sdr,c152_sdr,c160_sdr,c170_sdr,c180_sdr,c190_sdr,c192_sdr,c200_sdr,c210_sdr,c220_sdr,c230_sdr,c240_sdr,Location_Of_Defective_Part,Condition_Of_Defective_Part,c270_sdr,c280_sdr,c290_sdr,c300_sdr,c310a_sdr,c310b_sdr,c310c_sdr,c310d_sdr,...,c314c_sdr,c314d_sdr,c320a_sdr,c320b_sdr,c320c_sdr,c324a_sdr,c324b_sdr,c324c_sdr,c330_sdr,Stage_Of_Operation,c340_sdr,c350_sdr,c360_sdr,c370_sdr,c380_sdr,N_Number,c400_sdr,c410_sdr,c420_sdr,c430_sdr,c440_sdr,c450_sdr,c460_sdr,c490_sdr,c602_sdr,c604_sdr,c606_sdr,c608_sdr,c610_sdr,c612_sdr,c614_sdr,c616_sdr,c620_sdr,c640_sdr,c652_sdr,c654_sdr,c660_sdr,Remarks,Aircraft_Type,Keywords_sdr
0,SDR,2019-09-06,NaN,NM,2019,9,6,FDEA201904040764,1,2019-04-04,A,5320,ACA314011,FLOOR SUPPORT,NaN,NaN,DOUG,MD11,MD11F,3023600,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ZONE 200,CORRODED,A,NaN,NaN,FDEA,K,NaN,NaN,NaN,...,NaN,NaN,O,NaN,NaN,OTHER,NaN,NaN,IN,INSP/MAINT,1,NaN,NaN,CE,23,604FE,NaN,NaN,NaN,NaN,48460,NaN,NaN,NaN,NaN,NaN,NaN,3,4F,NaN,NaN,RT,A22WE,NaN,NaN,NaN,NaN,FLOOR BOARD SUPPORT CORRODED @ STA. 6-70 CENTE...,DOUG MD11,"floor,support,cabin,fabricated,structure,main,..."
1,SDR,2019-11-18,NaN,NE,2019,11,18,11FA2019070100001,2,2019-05-23,A,7200,NaN,ENGINE,NaN,NaN,CESSNA,510,510,2076617,CE,PWC,PW615,PW615FA,52282,NE,NaN,NaN,NaN,NaN,NR 2,MALFUNCTIONED,E,NaN,NaN,11FA,C,NaN,NaN,NaN,...,NaN,NaN,M,J,NaN,OVER TEMP,WARNING INDICATION,NaN,TO,TAKEOFF,1,NaN,NaN,SW,3,820UT,NaN,NaN,NaN,NaN,5100332,LB0666,NaN,NaN,NaN,NaN,NaN,2,4F,4,F,RT,A00014WI,E00073EN,NaN,NaN,NaN,ITT SPIKE NR 2 ENGINE EXCEEDED REDLINE. ENGINE...,CESSNA 510,"engine,fuel,pump,tank,idle,filter,blade"
2,SDR,2019-11-18,NaN,CE,2019,11,18,11FA2019070100002,1,2019-05-20,A,7722,NaN,UNKNOWN,NaN,NaN,CESSNA,510,510,2076617,CE,PWC,PW615,PW615FA,52282,NE,NaN,NaN,NaN,NaN,UNKNOWN,MALFUNCTIONED,E,NaN,NaN,11FA,K,NaN,NaN,NaN,...,NaN,NaN,J,M,NaN,WARNING INDICATION,OVER TEMP,NaN,CL,CLIMB,1,NaN,NaN,SW,3,820UT,NaN,NaN,NaN,NaN,5100332,LB0666,NaN,NaN,NaN,NaN,NaN,2,4F,4,F,RT,A00014WI,E00073EN,NaN,NaN,NaN,ITT SPIKE IN THE CLIMB FOR UNKNOWN REASONS. MA...,CESSNA 510,"engine,fuel,pump,tank,idle,filter,blade"
3,SDR,2020-03-24,NaN,NM,2020,3,24,120B201911200001,1,2019-11-18,A,2697,5534001R16,WIRE HARNESS,KIDDE,NA,BOEING,767,767323,1385205,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT FIRE LOOP,BROKEN,I,NaN,NaN,120B,K,NaN,NaN,NaN,...,NaN,NaN,O,NaN,NaN,OTHER,NaN,NaN,IN,INSP/MAINT,1,NaN,NaN,GL,3,36NE,NaN,NaN,NaN,NSN,25193,NaN,NaN,NaN,NaN,NaN,NaN,2,4F,NaN,NaN,RT,A1NM,NaN,NaN,NaN,NaN,RH ENGINE FIRE LOOP 1 DISPLAYED ON EICAS ON PR...,BOEING 767,"nbsp,start,mddr,eicas,message,status,span"
4,SDR,2020-06-02,NaN,NM,2020,6,2,120B201912240002,1,2019-12-23,A,2841,2021207,INDICATOR,NaN,NaN,BOEING,767,767323,1385205,NM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FUEL QTY,INOPERATIVE,I,NaN,NaN,120B,H,NaN,NaN,NaN,...,NaN,NaN,J,NaN,NaN,WARNING INDICATION,NaN,NaN,NR,NOT REPORTED,1,NaN,NaN,GL,3,225NE,99331,16,NaN,NaN,25194,NaN,NaN,NaN,NaN,NaN,NaN,2,4F,NaN,NaN,RT,A1NM,NaN,NaN,NaN,NaN,FUEL QUANTITY SYSTEM DISPLAYED ON PREFLIGHT EI...,BOEING 767,"fail,flaps,message,eicas,caution,radar,approach"


In [ ]:
eon_data1.rename(columns={'Dominant_Topic_eon': 'Keywords_eon'},inplace=True)
#Top 5 records of the final EON dataset
eon_data1.head()

,Report_Type,Unique_Identifier,Title,Description,Event_Type,Date_Of_Occurrence,Event Time,Reported By,Received Date,Latitude,Longitude,City,State,Airport,Region,Reporting Facility,Aircraft_Type,Departure,Arrival,Diverted To,POB,Injury Type,Injuries Minor,Injuries Serious,Fatalities,Injuries Unknown,Uninjured,POB Unknown,Damage Type,MessageText,Remarks,N_Number,Keywords_eon
0,EON,554891,MEDICAL EMERGENCY - PASSENGER | ASA1305 | SAN ...,NaN,MEDICAL EMERGENCY - PASSENGER,2019-01-01,02:01,NaN,NaN,37.618805,-122.375420,SAN FRANCISCO,CA,SFO,AWP,MOR/SFO @,A320,NaN,SFO,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"SAN FRANCISCO, CA (SFO)",SFODECLARED A MEDICAL EMERGENCY FOR A 29 YEAR ...,ASA1305,"destination,bird,damage,departing,strike,city,..."
1,EON,554892,GENERAL AVIATION OCCURRENCE | N9118C | SANTA R...,NaN,GENERAL AVIATION OCCURRENCE,2019-01-01,02:38,NaN,NaN,38.509693,-122.812889,SANTA ROSA,CA,STS,AWP,MOR/STS @ 1843 PST,C182,NaN,STS,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"SANTA ROSA, CA (STS)",STSLANDED WITHOUT INCIDENT AFTER REPORTING FAU...,9118C,"gear,indication,nose,warning,equipment,wheel,s..."
2,EON,554894,GENERAL AVIATION OCCURRENCE | N613PJ | TETERBO...,NaN,GENERAL AVIATION OCCURRENCE,2019-01-01,03:15,NaN,NaN,40.850113,-74.060837,TETERBORO,NJ,TEB,AEA,TEB,CL60,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"TETERBORO, NJ (TEB)",DIVERTED TO TEB AND LANDED WITHOUT INCIDENT AF...,613PJ,"failure,fuel,hydraulic,malfunction,alert,brake..."
3,EON,554895,"LASER EVENT | N4203P | PASCO, WA (PSC) 01-01-2...",NaN,LASER EVENT,2019-01-01,03:26,NaN,NaN,46.264694,-119.119026,PASCO,WA,PSC,ANM,MOR/PSC @ 1943 PST,C172,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"PASCO, WA (PSC)",WAS ILLUMINATED BY A GREEN LASER FROM THE 12 O...,4203P,"injuries,laser,green,illuminated,position,side..."
4,EON,554908,AIR CARRIER OR TAXI OCCURRENCE | NKS8618 | DET...,NaN,AIR CARRIER OR TAXI OCCURRENCE,2019-01-01,11:25,NaN,NaN,42.212444,-83.353386,DETROIT,MI,DTW,AGL,DTW MOR,A320,DTW,DFW,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,Not Reported,"DETROIT, MI (DTW)",DTW-DFW ABORTED TAKEOFF DUE TO A MALFUNCTION I...,NKS8618,"takeoff,taxied,ramp,back,running,parking,exited"


# Fuzzy Merging 

In [ ]:
pip install fuzzymatcher

     |████████████████████████████████| 51kB 2.9MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149806 sha256=9e5ae0c1046e307c4074d2539f73abe51f9131844f3adcfae22716ffbc963218
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
  Created wheel for metaphone: filename=Metaphone-0.6-cp37-none-any.whl size=13905 sha256=8fe7a28335e696e588f05503a0ac9f02afe4cde5c99c06e0086176a58ba5ad78
  Stored in directory: /root/.cache/pip/wheels/4e/7c/f7/162d726fc83491ef23c7a0b989005024eb83a1408c96f32eaf
Successfully built python-Levenshtein metaphone


In [ ]:
#Selecting AID dataset for Fuzzy Match
aid_fuzzy = aid_data1

#Selecting SDR dataset for Fuzzy Match
sdr_fuzzy = sdr_data1

#Selecting EON dataset for Fuzzy Match
eon_fuzzy = eon_data1

In [ ]:
#Renaming some of the AID columns to aid in matching process
aid_fuzzy.rename(columns={'Unique_Identifier':'Unique_Identifier_aid','Date_Of_Occurrence':'Date_Of_Occurrence_aid','Aircraft_Type':'Aircraft_Type_aid','Remarks':'Remarks_aid','Phase_Of_Flight':'Phase_Of_Flight_aid'}, inplace=True)

In [ ]:
#Renaming some of the SDR columns to aid in matching process
sdr_fuzzy.rename(columns={'Unique_Identifier':'Unique_Identifier_sdr','Date_Of_Occurrence':'Date_Of_Occurrence_sdr','Aircraft_Type':'Aircraft_Type_sdr','Remarks':'Remarks_sdr','Stage_Of_Operation':'Stage_Of_Operation_sdr'}, inplace=True)

In [ ]:
#Renaming some of the EON columns to aid in matching process
eon_fuzzy.rename(columns={'Date_Of_Occurrence':'Date_Of_Occurrence_eon','Aircraft_Type':'Aircraft_Type_eon','Remarks':'Remarks_eon','Unique_Identifier':'Unique_Identifier_eon'}, inplace=True)

In [ ]:
# Fuzzy Matching between AID and SDR

#importing the required library
import fuzzymatcher

#Columns considered for matching in the AID dataset
left_on = ["Date_Of_Occurrence_aid", "Aircraft_Type_aid", "Phase_Of_Flight_aid"]

#Columns considered for matching in the SDR dataset
right_on = ["Date_Of_Occurrence_sdr", "Aircraft_Type_sdr", "Stage_Of_Operation_sdr"]

#Applying the fuzzy_left_join function of the fuzzymatcher package
matched_results_aid_sdr = fuzzymatcher.fuzzy_left_join(aid_fuzzy,
                                              sdr_fuzzy,
                                              left_on,
                                              right_on,
                                              left_id_col='Unique_Identifier_aid',
                                              right_id_col='Unique_Identifier_sdr')

#Columns selected to have a overall view of the performance after fuzzy matching
cols = ["best_match_score", 'Unique_Identifier_aid', 'Unique_Identifier_sdr', "Date_Of_Occurrence_aid", "Date_Of_Occurrence_sdr", "Aircraft_Type_aid", "Aircraft_Type_sdr",
         "Phase_Of_Flight_aid", "Stage_Of_Operation_sdr","Remarks_aid","Remarks_sdr"]

#Top 20 records of the resulting matched records with the above columns sorted by best_match_score
matched_results_aid_sdr[cols].sort_values(by=['best_match_score'], ascending=False).head(20)

,best_match_score,Unique_Identifier_aid,Unique_Identifier_sdr,Date_Of_Occurrence_aid,Date_Of_Occurrence_sdr,Aircraft_Type_aid,Aircraft_Type_sdr,Phase_Of_Flight_aid,Stage_Of_Operation_sdr,Remarks_aid,Remarks_sdr
128554,0.814431,20190531005499I,CWQA201906040001,2019-05-31,2019-05-31,ISRAEL GALAXY,ISRAEL GALAXY,Cruise/Level Flight,CRUISE,"OWNER FLIGHT. DETROIT, MI (DTW): EJM540, GALX,...","DURING CRUISE, CREW RECEIVED BAGGAGE SMOKE EIC..."
189049,0.594333,20190913013099I,2019FA0000313,2019-09-13,2019-09-13,BEECH 65,BEECH 65,Takeoff: Climb Out,CLIMB,"ON SEPTEMBER 13, 2019 AT 16:15 EASTERN DAYLIGH...",INVESTIGATION REVEALED A BROKEN CLEVIS ON THE ...
74280,0.563857,20190330002179A,CA190401009,2019-03-30,2019-03-30,BELL 222,BELL 222,Takeoff: Climb Out,TAKEOFF,"ON MARCH 30, 2019 AT APPROXIMATELY 1230 EASTER...",(CAN) INITIAL REPORTS STATE A LOSS OF ENGINE P...
242959,0.492976,20190707009369I,2019FA0000222,2019-07-07,2019-07-11,PIPER PA46-500TP,PIPER PA46,Landing: Rollout,LANDING,AIRCRAFT LANDED MONTGOMERY FIELD (MYF) RUNWAY ...,CUSTOMER REPORTED ON LANDING ROLL LEFT MAIN GE...
381956,0.488292,20191113021819A,2019FA0000373,2019-11-13,2019-11-13,Unknown Aircraft,Unknown Aircraft,Maneuver,INSP/MAINT,CMH FSDO NOTIFIED BY OF ABC NEWS ON 11/22/2019...,WE WERE RECENTLY NOTIFIED OF A QUALITY ISSUE R...
29443,0.466209,20190318001619I,2020FA0000048,2019-03-18,2019-03-18,MOONEY M20E,MOONEY M20,Landing: Touchdown,INSP/MAINT,AFTER BEING CLEARED FOR A TOUCH AND GO ON RUNW...,THE BAGGAGE DOOR WAS LATCHED PRIOR TO FLIGHT. ...
244269,0.451191,20190916015719A,CA190729002,2019-09-16,2019-07-16,Unknown Aircraft,Unknown Aircraft,Takeoff: Climb Out,TAKEOFF,"ON OR ABOUT SEPTEMBER 16, 2019 AT APPROXIMATEL...",(CAN) CUSTOMER EXPERIENCED BOTH ENGINE SURGE W...
134604,0.445630,20190716011879I,PNSA2019071602,2019-07-16,2019-07-16,SAAB-S 2000,SAAB 2000,Cruise/Level Flight,CRUISE,07-16-19 AIRCRAFT N681PA OPERATED BY PENINSULA...,"LT ENGINE BRIEFLY WENT TO 0 PU'S 5SEC, ENGINE ..."
374935,0.441868,20190709021499A,CA190726003,2019-07-09,2019-07-09,Unknown Aircraft,Unknown Aircraft,Maneuver,CLIMB,"ON JULY 9, 2019 ABOUT 1732 EASTERN DAYLIGHT TI...",(CAN) DURING CLIMB AND TO PASS OVER THE CLOUDS...
51085,0.440824,20190524005729A,O2GA2019052801,2019-05-24,2019-05-24,CESSNA 560,CESSNA 560,Cruise/Level Flight,CRUISE,AIRCRAFT IN CRUISE AT FL 390. CHECKED IN WITH ...,"DURING LEVEL OFF AT FL22, 'PITCH TRIM' AMBER M..."


In [ ]:
matched_results_aid_sdr_SameDate = matched_results_aid_sdr[cols].sort_values(by=['best_match_score'], ascending=False).query('Date_Of_Occurrence_aid==Date_Of_Occurrence_sdr')
matched_results_aid_sdr_SameDate.to_csv('MatchedResults_AID-SDR_SameDate.csv', index=False)
matched_results_aid_sdr_SameDate

,best_match_score,Unique_Identifier_aid,Unique_Identifier_sdr,Date_Of_Occurrence_aid,Date_Of_Occurrence_sdr,Aircraft_Type_aid,Aircraft_Type_sdr,Phase_Of_Flight_aid,Stage_Of_Operation_sdr,Remarks_aid,Remarks_sdr
128554,0.814431,20190531005499I,CWQA201906040001,2019-05-31,2019-05-31,ISRAEL GALAXY,ISRAEL GALAXY,Cruise/Level Flight,CRUISE,"OWNER FLIGHT. DETROIT, MI (DTW): EJM540, GALX,...","DURING CRUISE, CREW RECEIVED BAGGAGE SMOKE EIC..."
189049,0.594333,20190913013099I,2019FA0000313,2019-09-13,2019-09-13,BEECH 65,BEECH 65,Takeoff: Climb Out,CLIMB,"ON SEPTEMBER 13, 2019 AT 16:15 EASTERN DAYLIGH...",INVESTIGATION REVEALED A BROKEN CLEVIS ON THE ...
74280,0.563857,20190330002179A,CA190401009,2019-03-30,2019-03-30,BELL 222,BELL 222,Takeoff: Climb Out,TAKEOFF,"ON MARCH 30, 2019 AT APPROXIMATELY 1230 EASTER...",(CAN) INITIAL REPORTS STATE A LOSS OF ENGINE P...
381956,0.488292,20191113021819A,2019FA0000373,2019-11-13,2019-11-13,Unknown Aircraft,Unknown Aircraft,Maneuver,INSP/MAINT,CMH FSDO NOTIFIED BY OF ABC NEWS ON 11/22/2019...,WE WERE RECENTLY NOTIFIED OF A QUALITY ISSUE R...
29443,0.466209,20190318001619I,2020FA0000048,2019-03-18,2019-03-18,MOONEY M20E,MOONEY M20,Landing: Touchdown,INSP/MAINT,AFTER BEING CLEARED FOR A TOUCH AND GO ON RUNW...,THE BAGGAGE DOOR WAS LATCHED PRIOR TO FLIGHT. ...
...,...,...,...,...,...,...,...,...,...,...,...
391904,-0.552890,20190821023009A,AALA20190821278,2019-08-21,2019-08-21,BURKHA G 103 TWIN I,BOEING 737,"Other, Specify",INSP/MAINT,"ON AUGUST 21, 2019 AT 1545 CDT, A GLIDER AIRCR...","R4 DOOR, EXTERIOR EMERGENCY LIGHT IS INOP"
7848,-0.557927,20190413023602A,ASAA2019041329127,2019-04-13,2019-04-13,MORRIS SPORTSMAN GS,BOEING 737,Run-Up Area,INSP/MAINT,"N3375G, A NORTH AMERICAN SNJ-5, TAXIED INTO N3...",FWD CARGO PIT R/H TRANSITION PANEL HAS MULTIPL...
152643,-0.588138,20190404010719I,FDEA201904040764,2019-04-04,2019-04-04,CONSOL LAKE LA-4,DOUG MD11,"Other, Specify",INSP/MAINT,PILOT STATED THE FIRST TAKEOFF WAS NORMAL THE ...,FLOOR BOARD SUPPORT CORRODED @ STA. 6-70 CENTE...
177680,-0.678469,20190508008339I,AALA20190508002,2019-05-08,2019-05-08,LX-7 L LX7-20A,BOEING 737,"Other, Specify",INSP/MAINT,PILOT OF N77VY WAS CONDUCTING A HIGH-SPEED TAX...,AIRCRAFT IN BASE MAINTENANCE: FORWARD WET ARE ...


In [ ]:
#drop some redundant columns to make matched results as a separate dataset to be used to match with EON
matched_results_aid_sdr = matched_results_aid_sdr.drop(matched_results_aid_sdr.columns[[0,1,2]],axis=1)

#Filtering the records with same date of occurrence
matched_results_aid_sdr_same_doc = matched_results_aid_sdr.query('Date_Of_Occurrence_aid==Date_Of_Occurrence_sdr')

#resetting the index of the matched resuts dataset
matched_results_aid_sdr_same_doc.reset_index(drop=True,inplace=True)

matched_results_aid_sdr_same_doc

,Report_Type_left,Unique_Identifier_aid,c1_aid,c2_aid,c3_aid,c4_aid,c6_aid,c7_aid,c8_aid,Date_Of_Occurrence_aid,c10_aid,c75_aid,c132_aid,c134_aid,c136_aid,c138_aid,c139_aid,c140_aid,c141_aid,c144_aid,c145_aid,c147_aid,c149_aid,c151_aid,c152_aid,c153_aid,c155_aid,c157_aid,c160_aid,c162_aid,c203_aid,c204_aid,c214_aid,c233_aid,c234_aid,c790_aid,N_Number_aid,Aircraft_Make_left,c24_aid,c25_aid,...,c314c_sdr,c314d_sdr,c320a_sdr,c320b_sdr,c320c_sdr,c324a_sdr,c324b_sdr,c324c_sdr,c330_sdr,Stage_Of_Operation_sdr,c340_sdr,c350_sdr,c360_sdr,c370_sdr,c380_sdr,N_Number,c400_sdr,c410_sdr,c420_sdr,c430_sdr,c440_sdr,c450_sdr,c460_sdr,c490_sdr,c602_sdr,c604_sdr,c606_sdr,c608_sdr,c610_sdr,c612_sdr,c614_sdr,c616_sdr,c620_sdr,c640_sdr,c652_sdr,c654_sdr,c660_sdr,Remarks_sdr,Aircraft_Type_sdr,Keywords_sdr
0,AID,20191013023229A,A,91,NaN,NaN,2019,10,13,2019-10-13,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CV,NaN,NaN,224306,2322,NaN,NaN,NaN,NaN,96709,TAYLOR,BC12-D,NaN,...,NaN,NaN,N,NaN,NaN,FALSE WARNING,NaN,NaN,TO,TAKEOFF,1,NaN,NaN,SW,21,568UW,NaN,NaN,NaN,NaN,5751,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RT,NaN,NaN,NaN,NaN,NaN,AIRCRAFT WAS GROUNDED: PILOT ABORTED TAKE OFF ...,AIRBUS A321,"wing,edge,repair,leading,beaded,doubler,slat"
1,AID,20191122023189A,A,91,NaN,NaN,2019,11,22,2019-11-22,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224301,2318,NaN,NaN,NaN,NaN,969JM,CIRRUS,SR22T,NaN,...,NaN,NaN,J,NaN,NaN,WARNING INDICATION,NaN,NaN,CR,CRUISE,1,NaN,NaN,NM,2,590AS,NaN,NaN,NaN,NaN,35687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4F,NaN,NaN,RT,NaN,NaN,NaN,NaN,NaN,AUTOTHROTTLE DISCONNECTED IN FLIGHT WOULD NOT ...,BOEING 737,"temp,epsu,recall,controller,master,zone,caution"
2,AID,20191125023199I,I,91,NaN,NaN,2019,11,25,2019-11-25,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224302,2319,NaN,NaN,NaN,NaN,3201L,MOONEY,M20J,NaN,...,NaN,NaN,O,NaN,NaN,OTHER,NaN,NaN,IN,INSP/MAINT,1,NaN,NaN,SW,21,944NN,NaN,NaN,NaN,NaN,31185,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4F,NaN,NaN,RT,NaN,NaN,NaN,NaN,NaN,AIRCRAFT IN BASE MAINTENANCE: CRACKED CABIN FL...,BOEING 737,"base,maintenance,transition,play,hinge,hand,el..."
3,AID,20191217023209A,A,91,NaN,NaN,2019,12,17,2019-12-17,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1L71,1,L,7,1,3O,NaN,NaN,TR,NaN,NaN,224303,2320,NaN,NaN,NaN,NaN,3113V,BEECH,35R,NaN,...,NaN,NaN,J,S,NaN,WARNING INDICATION,AFFECT SYSTEMS,NaN,CL,CLIMB,1,NaN,NaN,GL,27,530TM,10909,10909,NaN,NaN,RK289,NaN,NaN,NaN,NaN,NaN,NaN,2,4F,NaN,NaN,RT,A16SW,NaN,NaN,NaN,NaN,PIC REPORTS LANDING GEAR TAKES 90 SECONDS TO R...,BEECH MU300,"gear,landing,takeoff,nose,take,disagree,extension"
4,AID,20191218023219I,I,91,NaN,NaN,2019,12,18,2019-12-18,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,224304,2321,NaN,NaN,NaN,NaN,6919J,PIPER,PA-28-151,NaN,...,NaN,NaN,J,NaN,NaN,WARNING INDICATION,NaN,NaN,LD,LANDING,1,NaN,NaN,AL,3,4107Q,NaN,NaN,NaN,NaN,318253008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,NaN,O,5,NaN,P33EA,NaN,NaN,LH MLG RED LIGHT ON WHEN GEAR SUPPOSED TO BE D...,PIPER PA31,"smell,odor,pack,cockpit,smoke,packs,fumes"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,AID,20190921023099I,I,91,NaN,NaN,2019,9,21,2019-09-21,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CV,NaN,NaN,224291,2309,NaN,NaN,NaN,NaN,21BW,BLESSI,PITTS S1-C,NaN,...,NaN,NaN,N,NaN,NaN,FALSE WARNING,NaN,NaN,LD,LANDING,1,NaN,NaN,NM,2,461AS,NaN,NaN,NaN,NaN,36363,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4F,NaN,NaN,RT,NaN,NaN,NaN,NaN,NaN,VTK FLAGS APPEARED ON BOTH L AND R DISPLAYS AF...,BOEING 737,"lens,cover,missing,installed,exit,megaphone,di..."
1091,AID,20190921023109I,I,91,NaN,NaN,2019,9,21,2019-09-21,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TR,NaN,NaN,

In [ ]:
# Fuzzy Matching between results of AID-SDR with EON using columns of SDR

#Columns considered in the matched results dataset of aid-sdr for matching
left_on = ["Date_Of_Occurrence_sdr", "Aircraft_Type_sdr"]

#Columns considered in the EON dataset for matching
right_on = ["Date_Of_Occurrence_eon", "Aircraft_Type_eon"]

#Implementing fuzzy left join with matched results of aid_sdr on EON
matched_results_aid_sdr_eon = fuzzymatcher.fuzzy_left_join(matched_results_aid_sdr_same_doc,
                                            eon_fuzzy,
                                            left_on,
                                            right_on,
                                            left_id_col='Unique_Identifier_sdr',
                                            right_id_col='Unique_Identifier_eon')

#Columns selected to have a overall view of the performance after fuzzy matching
cols = ["best_match_score", 'Unique_Identifier_aid','Unique_Identifier_sdr', 'Unique_Identifier_eon', "Date_Of_Occurrence_aid", 'Date_Of_Occurrence_sdr',"Date_Of_Occurrence_eon", "Aircraft_Type_aid",'Aircraft_Type_sdr', "Aircraft_Type_eon",
        "Remarks_aid",'Remarks_sdr',"Remarks_eon"]

#Top 20 records of the resulting matched records with the above columns sorted by best_match_score
matched_results_aid_sdr_eon[cols].sort_values(by=['best_match_score'], ascending=False).head(10)

,best_match_score,Unique_Identifier_aid,Unique_Identifier_sdr,Unique_Identifier_eon,Date_Of_Occurrence_aid,Date_Of_Occurrence_sdr,Date_Of_Occurrence_eon,Aircraft_Type_aid,Aircraft_Type_sdr,Aircraft_Type_eon,Remarks_aid,Remarks_sdr,Remarks_eon
174027,0.344321,20190306022499A,CA190311024,573325,2019-03-06,2019-03-06,2019-03-23,CESSNA 172,CESSNA 172,CESSNA 172,"PILOT TOOK OFF FROM SUBMARINE LAKE, NEAR FAREW...",(CAN) DURING THE FLIGHT THE PILOT REPORTED SMO...,MADE A SUDDEN RIGHT TURN OF 30 DEGREES RIGHT I...
75166,0.343954,20190309010659A,2019FA0000115,573325,2019-03-09,2019-03-09,2019-03-23,CESSNA T210M,CESSNA 172,CESSNA 172,AIRMAN WAS DISTRACTED ON DOWNWIND SEARCHING FO...,A/C WAS INVOLVED IN AN ACCIDENT AT VAY. UPON L...,MADE A SUDDEN RIGHT TURN OF 30 DEGREES RIGHT I...
26924,0.326237,20190313001299A,2019FA0000069,591492,2019-03-13,2019-03-13,2019-08-13,BELL OH-58A,BELL 206,BELL 206,"ON MARCH 13, 2019 THE PIC WAS ASSIGNED TO PERF...",DURING AN OAS INSPECTION A SMALL CRACK WAS FOU...,N88TA ACC/INC @ OLYMPIA WA MADE EMERG OFF ARP...
95810,0.325335,20190321010681A,CA190325012,573325,2019-03-21,2019-03-21,2019-03-23,CESSNA 172M,CESSNA 172,CESSNA 172,"HOLLYWOOD, FL (HWO): N20521, C172, STRUCK N704...",(CAN) WHILE PERFORMING PRE-FLIGHT ENGINE RUN U...,MADE A SUDDEN RIGHT TURN OF 30 DEGREES RIGHT I...
22742,0.277040,20190320000569A,2020FA0000127,567104,2019-03-20,2019-03-20,2019-03-20,ROBINS R22 BETA,ROBSIN R22,R22,"PILOT TOOK OFF FROM MADILL, OK (IF4) HEADING T...",CLUTCH BELTS BROKE IN FLIGHT. REASON UNKNOWN.,EXPERIENCED CLUTCH BELT (BOTH) FAILURE AND AUT...
90044,0.275041,20190802013229A,YV3R2019081920164,589804,2019-08-02,2019-08-02,2019-08-02,ROBINS R66,ROBSIN R66,R66,"ON AUGUST 02, 2019 AT 1535 LOCAL TIME A ROBINS...",CUSTOMER REPORTED DUAL TACHOMETER P/N: F792-1 ...,DURING AERIAL APPLICATION IMPACTED ELECTRICAL ...
40073,0.272800,20190415002849A,QMLA2019041759037,572527,2019-04-15,2019-04-15,2019-04-25,BELL H 206L-1,BELL 407,BELL 407,"ON APRIL 15, 2019 AT APPROXIMATELY 0345, N395A...",4 MINUTES AFTER TAKEOFF ALL CREW MENBERS NOTIC...,E 3 N105AM REPORTED A WHITE UAS AT THE 12 O'C...
124466,0.269203,20190417010829A,TLLA20190417001,571361,2019-04-17,2019-04-17,2019-04-16,BELL 47G-3B-1,BELL 206,BELL 206,A BELL 47 AIRCRAFT BEARING REGISTRATION NUMBER...,DURING SCHEDULED INSPECTION WORKING RIVETS WER...,CRASHED UNKN CIRCUMSTANCES. POB
172808,0.264422,20191214021429A,YV3R2020012020222,608044,2019-12-14,2019-12-14,2019-12-14,ROBINS R44 II,ROBSIN R44,R44,"ON DECEMBER 14, 2019 AT 1450 CST. N440AH A ROB...",CUSTOMER REPORTED V-BELT SET P/N: A190-3 SNAPPED.,"44, GREEN LASER FROM THE 12 O'CLOCK POSITION W..."
47432,0.261943,20190621007739A,CA190622005,582350,2019-06-21,2019-06-21,2019-06-21,ROBINS R44 II,ROBSIN R44,R44,PILOT MAKING LAST SWATH OF THE DAY SPRAYING CO...,"(CAN) DURING WARM UP OF ENGINE, PILOT NOTED AN...","4, OBSERVED A WHITE QUADCOPTER UAS AT 350 FEET..."


In [ ]:
matched_results_aid_sdr_eon_SameDate = matched_results_aid_sdr_eon[cols].sort_values(by=['best_match_score'], ascending=False).query('Date_Of_Occurrence_aid==Date_Of_Occurrence_sdr==Date_Of_Occurrence_eon')
matched_results_aid_sdr_eon_SameDate.to_csv('MatchedResults_AID-SDR-EON_SameDate.csv', index=False)
matched_results_aid_sdr_eon_SameDate

,best_match_score,Unique_Identifier_aid,Unique_Identifier_sdr,Unique_Identifier_eon,Date_Of_Occurrence_aid,Date_Of_Occurrence_sdr,Date_Of_Occurrence_eon,Aircraft_Type_aid,Aircraft_Type_sdr,Aircraft_Type_eon,Remarks_aid,Remarks_sdr,Remarks_eon
22742,0.277040,20190320000569A,2020FA0000127,567104,2019-03-20,2019-03-20,2019-03-20,ROBINS R22 BETA,ROBSIN R22,R22,"PILOT TOOK OFF FROM MADILL, OK (IF4) HEADING T...",CLUTCH BELTS BROKE IN FLIGHT. REASON UNKNOWN.,EXPERIENCED CLUTCH BELT (BOTH) FAILURE AND AUT...
90044,0.275041,20190802013229A,YV3R2019081920164,589804,2019-08-02,2019-08-02,2019-08-02,ROBINS R66,ROBSIN R66,R66,"ON AUGUST 02, 2019 AT 1535 LOCAL TIME A ROBINS...",CUSTOMER REPORTED DUAL TACHOMETER P/N: F792-1 ...,DURING AERIAL APPLICATION IMPACTED ELECTRICAL ...
172808,0.264422,20191214021429A,YV3R2020012020222,608044,2019-12-14,2019-12-14,2019-12-14,ROBINS R44 II,ROBSIN R44,R44,"ON DECEMBER 14, 2019 AT 1450 CST. N440AH A ROB...",CUSTOMER REPORTED V-BELT SET P/N: A190-3 SNAPPED.,"44, GREEN LASER FROM THE 12 O'CLOCK POSITION W..."
47432,0.261943,20190621007739A,CA190622005,582350,2019-06-21,2019-06-21,2019-06-21,ROBINS R44 II,ROBSIN R44,R44,PILOT MAKING LAST SWATH OF THE DAY SPRAYING CO...,"(CAN) DURING WARM UP OF ENGINE, PILOT NOTED AN...","4, OBSERVED A WHITE QUADCOPTER UAS AT 350 FEET..."
3200,0.261891,20190716023559A,YV3R2019080220150,588007,2019-07-16,2019-07-16,2019-07-16,ROBINS R44 II,ROBSIN R44,R44,RECEIVED A PHONE CALL FROM THE REGIONAL OPERAT...,CUSTOMER REPORTED DUAL TACHOMETER P/N: C792-4 ...,EVIATION REPORTED BY RSW WHEN N251JS DEPARTED...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116538,-0.192641,20190306015589A,SWAA2019030730406,564221,2019-03-06,2019-03-06,2019-03-06,CESSNA 172,BOEING 737,B738,"PILOT WAS FLYING VFR FROM SUBMARINE LAKE, AK. ...",L1 DOOR EMERGENCY EXIT LIGHT COVER DETACHED. R...,KLAS CONFIGURATION ONE VFR PILOT REJECTED TAKE...
78518,-0.194442,20190607006149I,SWAA2019060733450,579191,2019-06-07,2019-06-07,2019-06-07,DUBOIS COZY,BOEING 737,B738,"ON JUNE 7, 2019, AN EXPERIMENTAL AMATEUR BUILT...",DECLARED EMERGENCY. FLAPS DID NOT DEPLOY FOR L...,REPORTED HAVING A FLAPS ISSUE CRASH PHONE RUNG...
77550,-0.194442,20190607006069I,SWAA2019060733450,579191,2019-06-07,2019-06-07,2019-06-07,NANCHA CJ-6A,BOEING 737,B738,PILOT WAS LANDING AT KTOA ON RUNWAY 29R AND TH...,DECLARED EMERGENCY. FLAPS DID NOT DEPLOY FOR L...,REPORTED HAVING A FLAPS ISSUE CRASH PHONE RUNG...
78721,-0.194442,20190607006159I,SWAA2019060733450,579191,2019-06-07,2019-06-07,2019-06-07,MOONEY M20F,BOEING 737,B738,WHILE FLYING A NEWLY PURCHASED M20F FROM SOUTH...,DECLARED EMERGENCY. FLAPS DID NOT DEPLOY FOR L...,REPORTED HAVING A FLAPS ISSUE CRASH PHONE RUNG...


In [ ]:
# Fuzzy Matching between AID and EON

#Columns considered for matching in the AID dataset
left_on = ["Date_Of_Occurrence_aid", "Aircraft_Type_aid"]

#Columns considered for matching in the EON dataset
right_on = ["Date_Of_Occurrence_eon", "Aircraft_Type_eon"]

#Applying the fuzzy_left_join function of the fuzzymatcher package
matched_results_aid_eon = fuzzymatcher.fuzzy_left_join(aid_fuzzy,
                                            eon_fuzzy,
                                            left_on,
                                            right_on,
                                            left_id_col='Unique_Identifier_aid',
                                            right_id_col='Unique_Identifier_eon')

#Columns selected to have a overall view of the performance after fuzzy matching
cols = ["best_match_score", 'Unique_Identifier_aid', 'Unique_Identifier_eon', "Date_Of_Occurrence_aid", "Date_Of_Occurrence_eon", "Aircraft_Type_aid", "Aircraft_Type_eon",
        "Remarks_aid","Remarks_eon"]

#Top 20 records of the resulting matched records with the above columns sorted by best_match_score
matched_results_aid_eon[cols].sort_values(by=['best_match_score'], ascending=False).head(20)

,best_match_score,Unique_Identifier_aid,Unique_Identifier_eon,Date_Of_Occurrence_aid,Date_Of_Occurrence_eon,Aircraft_Type_aid,Aircraft_Type_eon,Remarks_aid,Remarks_eon
5202,1.050865,20190413023612A,571042,2019-04-13,2019-04-13,NORTH SNJ-5,NORTH AMERICAN SNJ-5,"N3375G, A NORTH AMERICAN SNJ-5, TAXIED INTO N3...",CNOWHILE N316RM WAS IN RUN-UP AREA FOR RWY 26R...
51335,0.968845,20190611006649A,579793,2019-06-11,2019-06-11,MAULE MX-7-180A,MAULE MX-7-180A,"ON JUNE 11, 2019 AT 1115 AKDT, N61339,A MAULE ...",L85ATTEMPTED WATER LANDING (MACKEYS LAKES SEAP...
141607,0.930696,20190622007809A,582452,2019-06-22,2019-06-22,MAULE M-7-235B,MAULE M-7-235B,ARRIVED ON SCENE APPROX. 1930 LOCAL. LOCATED D...,S21AFTER DEPARTING SUNRIVER (S21) AIRCRAFT WAS...
274956,0.906805,20191224018889A,612425,2019-12-24,2019-12-24,WELLS WELLS ROCKET,WELLS ROCKET,WHILE PERFORMING LIGHT AEROBATICS A LOUD NOISE...,DEPARTED AND RETURNED TO PVG DUE TO FLIGHT CON...
228614,0.883622,20191007014389A,599109,2019-10-07,2019-10-07,HONDA HA-420,HONDA HA-420,N166HJ WAS ON APPROACH TO CHARLESTON AIRPORT (...,"5, INJURIES NONE, DAMAGESUBSTANTIAL. 10/07/201..."
52773,0.861512,20190730006739A,589259,2019-07-30,2019-07-30,WILLIA AVID MARK IV,AVID MARK IV,DESCRIPTION OF THE EVENT: AIRCRAFT OWNER/BUILD...,CRASHED UNKN CIRC PRVT STRIP @ 920 BLUE SLIDE ...
322041,0.856793,20191109022939A,603735,2019-11-09,2019-11-09,BEECH 95-C55,BEECH 95-C55,THE PIC WAS HAVING A CHECK RIDE. THE PIC DEPAR...,BJC-BJC DURING A PLANNED TOUCH-AND-GO THE AIRC...
238513,0.843846,20190707015509A,584690,2019-07-07,2019-07-08,NORTH SCOUT X-C,NORTH WING UUM SCOUT X-C,"ON JULY 7, 2019, AT APPROXIMATELY 2055 EASTERN...",WEIGHT-SHIFT CONTROL MADE A HARD LANDING ON TU...
243587,0.782422,20191122017249I,579793,2019-11-22,2019-06-11,MAULE MX-7-180A,MAULE MX-7-180A,PILOT GROUND LOOPED ON LANDING AND SCRAPED LEF...,L85ATTEMPTED WATER LANDING (MACKEYS LAKES SEAP...
267069,0.781877,20190918018189I,596576,2019-09-18,2019-09-18,BEECH 35-33,BEECH 35-33,"N939L, A BEECHCRAFT 35-33, LANDED RIVERSIDE MU...",RALLEFT MAIN AND NOSE GEAR COLLAPSED AFTER A/C...


In [ ]:
matched_results_aid_eon_SameDate = matched_results_aid_eon[cols].sort_values(by=['best_match_score'], ascending=False).query('Date_Of_Occurrence_aid==Date_Of_Occurrence_eon')
matched_results_aid_eon_SameDate.to_csv('MatchedResults_AID-EON_SameDate.csv', index=False)
matched_results_aid_eon_SameDate

,best_match_score,Unique_Identifier_aid,Unique_Identifier_eon,Date_Of_Occurrence_aid,Date_Of_Occurrence_eon,Aircraft_Type_aid,Aircraft_Type_eon,Remarks_aid,Remarks_eon
5202,1.050865,20190413023612A,571042,2019-04-13,2019-04-13,NORTH SNJ-5,NORTH AMERICAN SNJ-5,"N3375G, A NORTH AMERICAN SNJ-5, TAXIED INTO N3...",CNOWHILE N316RM WAS IN RUN-UP AREA FOR RWY 26R...
51335,0.968845,20190611006649A,579793,2019-06-11,2019-06-11,MAULE MX-7-180A,MAULE MX-7-180A,"ON JUNE 11, 2019 AT 1115 AKDT, N61339,A MAULE ...",L85ATTEMPTED WATER LANDING (MACKEYS LAKES SEAP...
141607,0.930696,20190622007809A,582452,2019-06-22,2019-06-22,MAULE M-7-235B,MAULE M-7-235B,ARRIVED ON SCENE APPROX. 1930 LOCAL. LOCATED D...,S21AFTER DEPARTING SUNRIVER (S21) AIRCRAFT WAS...
274956,0.906805,20191224018889A,612425,2019-12-24,2019-12-24,WELLS WELLS ROCKET,WELLS ROCKET,WHILE PERFORMING LIGHT AEROBATICS A LOUD NOISE...,DEPARTED AND RETURNED TO PVG DUE TO FLIGHT CON...
228614,0.883622,20191007014389A,599109,2019-10-07,2019-10-07,HONDA HA-420,HONDA HA-420,N166HJ WAS ON APPROACH TO CHARLESTON AIRPORT (...,"5, INJURIES NONE, DAMAGESUBSTANTIAL. 10/07/201..."
...,...,...,...,...,...,...,...,...,...
207588,-0.386682,20191005019029A,599288,2019-10-05,2019-10-05,REED M SMITHS SUPER,C172,"ON OCTOBER 5, 2019, AT APPROXIMATELY 0915 LOCA...",NOT REPORTED
303900,-0.401143,20190209022309I,560307,2019-02-09,2019-02-09,FOCKE FW 190A-9,C172,"N190RF, A FW-190 ENTERED THE PATTERN AT CHINO ...",CESSNA IN THE PATTERN FOR RWY 22R TURNING DOWN...
92586,-0.418803,20190427004529A,572804,2019-04-27,2019-04-27,HORNBE ZODIAC CH 60,C172,"PILOT WAS FLYING N61PG FROM BQ1 CARTHAGE, NC T...",REPORTED A BIRD STRIKE LANDING RUNWAY 23R. AIR...
320450,-0.436315,20191119022689I,604972,2019-11-19,2019-11-19,FORD W STARDUSTER S,C172,"ON NOVEMBER 19, 2019 AT 1238 EDT A 1981 FORD W...",N54202 REPORTED A GREEN LASER ILLUMINATION WH...


In [ ]:
# Fuzzy Matching between SDR and EON

import fuzzymatcher

#Columns considered for matching in the SDR dataset
right_on = ["Date_Of_Occurrence_sdr", "Aircraft_Type_sdr"]

#Columns considered for matching in the EON dataset
left_on = ["Date_Of_Occurrence_eon", "Aircraft_Type_eon"]


matched_results_eon_sdr = fuzzymatcher.fuzzy_left_join(eon_fuzzy,
                                            sdr_fuzzy,
                                            left_on,
                                            right_on,
                                            left_id_col='Unique_Identifier_eon',
                                            right_id_col='Unique_Identifier_sdr')

cols = ["best_match_score", 'Unique_Identifier_eon', 'Unique_Identifier_sdr', "Date_Of_Occurrence_eon", "Date_Of_Occurrence_sdr", "Aircraft_Type_eon", "Aircraft_Type_sdr",
        "Remarks_eon","Remarks_sdr"]

matched_results_eon_sdr[cols].sort_values(by=['best_match_score'], ascending=False).head(10)

,best_match_score,Unique_Identifier_eon,Unique_Identifier_sdr,Date_Of_Occurrence_eon,Date_Of_Occurrence_sdr,Aircraft_Type_eon,Aircraft_Type_sdr,Remarks_eon,Remarks_sdr
1006838,0.390058,582758,2019FA0000210,2019-06-21,2019-06-21,PIPER L-21B,PIPER PA28,1DA-OR9 AIRCRAFT GROUND LOOPED ON TAKEOFF AT L...,WHILE RETENSIONING THE LOWER STABILATOR TURN B...
2536217,0.336448,608816,D7TA201912230001,2019-12-20,2019-12-20,DC3,DOUG DC3,N3006 REPORTED A GREEN LASER ILLUMINATION WHI...,UPON CLIMB - PASSING THROUGH 8500 FEET. THE FI...
1238018,0.334454,587036,HEER20190507882,2019-07-07,2019-05-07,BELL 407,BELL 407,EVIATION REPORTED BY LAW ATCT WHEN N390SF (SU...,FORWARD SHELL ASSEMBLY INNER SKIN WAS FOUND TO...
1030812,0.324931,582641,W6JA201906254036,2019-06-24,2019-06-24,G280,GULSTM G280,GLR-CAE REPORTED A STEERING ISSUE. AIRCRAFT LA...,AIRCRAFT EXPERIENCED A YAW DAMP DISENGAGED IN ...
713245,0.322211,571237,2019FA0000119,2019-04-15,2019-04-15,PA24,PIPER PA24,DVT-DVT RETURNED TO DVT AND LANDED WITHOUT INC...,INTERNAL CORROSION OF THE ENGINE MOUNT ASSEMBL...
818234,0.322211,574012,2019FA0000119,2019-04-15,2019-04-15,PA24,PIPER PA24,EVIATION REPORTED BY DVT WHEN N6364P THE PILO...,INTERNAL CORROSION OF THE ENGINE MOUNT ASSEMBL...
332391,0.320515,561958,2019FA0000184,2019-02-24,2019-02-24,PA23,PIPER PA23,"A23, ICTVFR, PILOT REPORTED NO GREEN ON MAIN L...",DURING INSPECTION TO DETERMINE CAUSE OF LANDIN...
2290484,0.319972,604470,CA190219023,2019-11-15,2019-02-15,BELL 212,BELL 212,COCKPIT WAS ILLUMINATED BY A GREEN LASER AT TH...,(CAN) TRD OUTER COUPLING FOUND WITH POSSIBLE M...
1490471,0.317561,591559,2019FA0000282,2019-08-14,2019-08-14,B25,NAMER B25,CPS-FSD REPORTED ENGINE SHUTDOWN AND RETURNED ...,DURING POST FLIGHT INSPECTION FOR A RELATED IS...
154311,0.314630,560095,O2GA2019020188901,2019-01-21,2019-01-21,B300,BEECH B300,EVIATION REPORTED BY JCF WHEN N512DW AIRCRAFT...,CREW REPORTED AFTER POST FLIGHT NOTED ALL 4 RI...


In [ ]:
matched_results_eon_sdr_SameDate = matched_results_eon_sdr[cols].sort_values(by=['best_match_score'], ascending=False).query('Date_Of_Occurrence_sdr==Date_Of_Occurrence_eon')
matched_results_eon_sdr_SameDate.to_csv('MatchedResults_EON-SDR_SameDate.csv', index=False)
matched_results_eon_sdr_SameDate

,best_match_score,Unique_Identifier_eon,Unique_Identifier_sdr,Date_Of_Occurrence_eon,Date_Of_Occurrence_sdr,Aircraft_Type_eon,Aircraft_Type_sdr,Remarks_eon,Remarks_sdr
1006838,0.390058,582758,2019FA0000210,2019-06-21,2019-06-21,PIPER L-21B,PIPER PA28,1DA-OR9 AIRCRAFT GROUND LOOPED ON TAKEOFF AT L...,WHILE RETENSIONING THE LOWER STABILATOR TURN B...
2536217,0.336448,608816,D7TA201912230001,2019-12-20,2019-12-20,DC3,DOUG DC3,N3006 REPORTED A GREEN LASER ILLUMINATION WHI...,UPON CLIMB - PASSING THROUGH 8500 FEET. THE FI...
1030812,0.324931,582641,W6JA201906254036,2019-06-24,2019-06-24,G280,GULSTM G280,GLR-CAE REPORTED A STEERING ISSUE. AIRCRAFT LA...,AIRCRAFT EXPERIENCED A YAW DAMP DISENGAGED IN ...
713245,0.322211,571237,2019FA0000119,2019-04-15,2019-04-15,PA24,PIPER PA24,DVT-DVT RETURNED TO DVT AND LANDED WITHOUT INC...,INTERNAL CORROSION OF THE ENGINE MOUNT ASSEMBL...
818234,0.322211,574012,2019FA0000119,2019-04-15,2019-04-15,PA24,PIPER PA24,EVIATION REPORTED BY DVT WHEN N6364P THE PILO...,INTERNAL CORROSION OF THE ENGINE MOUNT ASSEMBL...
...,...,...,...,...,...,...,...,...,...
1241793,-0.576366,587095,AALA201907179004,2019-07-16,2019-07-16,ROCKWELL AIR TRACTOR S-2R,BOEING 737,ACFT FORCED LANDED IN FIELD DUE TO MALFUNCTION...,AIRCRAFT WAS NOT GROUNDED: L1 DOOR SLIDE WAS L...
1288854,-0.592959,587913,AALA201907229006,2019-07-21,2019-07-21,OR PD DISPATCH/RICH HAY,BOEING 737,"SHERMAN, PILOT HIT POWER LINES AND A POWER LIN...",AIRCRAFT WAS NOT GROUNDED: ON PREFLIGHT FOUND ...
837206,-0.621497,576529,AALA201905279006,2019-05-25,2019-05-25,CA POLICE/530-823-4232,BOEING 737,"STINSON 108-2, AUN-AUN, INSTRUCTOR AND STUDENT...",AIRCRAFT WAS NOT GROUNDED: FLIGHT ATTENDANT EM...
1169281,-0.748837,584690,AALA20190709001,2019-07-08,2019-07-08,NORTH WING UUM SCOUT X-C,BOEING 737,WEIGHT-SHIFT CONTROL MADE A HARD LANDING ON TU...,AIRCRAFT IN BASE MAINTENANCE: FORWARD WET AREA...
